## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-08-49-48 +0000,nyt,What Europe Learned From the Greenland Crisis,https://www.nytimes.com/2026/01/24/world/europ...
1,2026-01-24-08-48-40 +0000,bbc,Ukraine condemns 'brutal' Russian strikes ahea...,https://www.bbc.com/news/articles/cwyr6p30054o...
2,2026-01-24-08-15-18 +0000,nypost,FDNY firefighters battle 4-alarm blaze on top ...,https://nypost.com/2026/01/24/us-news/fdny-fir...
3,2026-01-24-08-15-08 +0000,nyt,"Extreme Cold Sweeps Canada, With Manitoba and ...",https://www.nytimes.com/2026/01/23/weather/can...
4,2026-01-24-07-57-00 +0000,bbc,Pentagon to offer 'more limited' support to US...,https://www.bbc.com/news/articles/cj9r8ezym3ro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2427/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
208,trump,55
130,ice,22
2,greenland,17
67,new,11
522,russia,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
285,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...,98
44,2026-01-24-02-00-00 +0000,wsj,Trump’s aggressive deployment of ICE has thrus...,https://www.wsj.com/politics/policy/democrat-r...,97
241,2026-01-23-15-04-40 +0000,wapo,"On Greenland, Europe stood up, Trump blinked, ...",https://www.washingtonpost.com/world/2026/01/2...,94
206,2026-01-23-17-00-09 +0000,latimes,Trump lawyers urge Supreme Court to block Cali...,https://www.latimes.com/politics/story/2026-01...,92
136,2026-01-23-21-10-44 +0000,nyt,U.K.’s Starmer Calls Trump’s Claims About NATO...,https://www.nytimes.com/2026/01/23/world/europ...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
285,98,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...
14,59,2026-01-24-05-31-24 +0000,startribune,"The latest: Thousands march, scores of Minneso...",https://www.startribune.com/ice-raids-minnesot...
80,39,2026-01-23-23-34-51 +0000,nypost,Ex-NJ Gov. Phil Murphy granted clemency to mom...,https://nypost.com/2026/01/23/us-news/murphy-g...
217,38,2026-01-23-16-18-59 +0000,nypost,NYC judge paves way for delivery customers to ...,https://nypost.com/2026/01/23/business/judge-d...
46,35,2026-01-24-01-39-26 +0000,nypost,Shocking question 4-year-old daughter of fetis...,https://nypost.com/2026/01/23/us-news/shocking...
130,35,2026-01-23-21-39-00 +0000,wsj,Intel Shares Slide as Costs Pile Up in Bid to ...,https://www.wsj.com/business/earnings/intel-in...
21,32,2026-01-24-04-00-00 +0000,wsj,China Built a Vast Oil Stake in Venezuela. Now...,https://www.wsj.com/world/americas/china-venez...
220,31,2026-01-23-16-11-32 +0000,wapo,Olympic snowboarder turned ‘Most Wanted’ fugit...,https://www.washingtonpost.com/national-securi...
136,31,2026-01-23-21-10-44 +0000,nyt,U.K.’s Starmer Calls Trump’s Claims About NATO...,https://www.nytimes.com/2026/01/23/world/europ...
221,30,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
